In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import InMemoryConversationMemory, JSONConversationMemory
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
from langrila import ToolConfig, ToolParameter, ToolProperty

tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is set! The disco ball is spinning, energetic music is playing at a lively BPM of 120, and the lights are dimmed to create the perfect atmosphere. Get ready to have a blast! 🎉💃🕺')], name=None), usage=Usage(prompt_tokens=134, completion_tokens=50, total_tokens=184), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_pVTN0uEHDlp3o8YuWJw5O20m', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_j6LSRIIiW1AvBDMFDGmIP8np', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_Z9Cuy2DkgziV9ZRdrjM8B28q', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, 

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively BPM of 120, and the lights are dimmed to create the perfect atmosphere. Get ready to have a blast! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 134,
  'completion_tokens': 50},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pVTN0uEHDlp3o8YuWJw5O20m',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_j6LSRIIiW1AvBDMFDGmIP8np',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_Z9Cuy2DkgziV9ZRdrjM8B28q',
     'function': {'a

Asynchronous generation

In [10]:
response = await chat_openai.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The atmosphere is coming alive! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect party vibe. Get ready to have some fun! 🕺💃✨')], name=None), usage=Usage(prompt_tokens=134, completion_tokens=49, total_tokens=183), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_d1q8hbTvIH3U3Z6bt3ZbpLjJ', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_MAbVoIDpEF3YZLu8jHOI6V79', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_EjhtbpY3m3BchWy1Rxc2eXHa', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'functio

multiple generation

In [11]:
response = chat_openai.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party atmosphere is set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed for a cozy vibe. Get ready to have a great time! 🎉💃🕺'},
  {'text': 'The party mood is on! The disco ball is spinning, energetic music is playing loudly, and the lights are dimmed to create a cozy atmosphere. Let the festivities begin! 🎉✨'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chat_openai.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've started some energetic music to set the mood! Now, let's add some more elements to create a comfortable party atmosphere:\n\n1. **Lighting**: Dim the lights and add some colorful LED string lights or fairy lights for a festive vibe. If you have any disco or party lights, this is the time to use them!\n\n2. **Seating Arrangements**: Arrange comfortable seating with cushions and throws. Create small gathering spots for groups to mingle.\n\n3. **Refreshments**: Set up a drinks station with a variety of beverages—cocktails, mocktails, and soft drinks. Add some snacks like chips, popcorn, and finger foods.\n\n4. **Games or Activities**: Consider incorporating some fun party games or activities that guests can enjoy. This could include card games, a photo booth, or even just a dance floor!\n\n5. **Scent**: Use scented candles or diffusers with light, refreshing fragrances to enhance the atmosphere and make it feel inviting.\n\n6.

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chat_openai.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_wcdOyaWf388wBhnyhjXWComo', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights'), Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_cfxmX91X6Cibza7GFed2l3aI', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_o6EBAFhc9xa3qvuMwSRe5s4c', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music')], calls=Message(role='function_call', content=[ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_wcdOyaWf388wBhnyhjXWComo'), ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_cfxmX91X6Cibza7GFed2l3aI'), To

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 171,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_wcdOyaWf388wBhnyhjXWComo',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'},
  {'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_cfxmX91X6Cibza7GFed2l3aI',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_o6EBAFhc9xa3qvuMwSRe5s4c',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_wcdOyaWf388wBhnyhjXWComo'},


In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_wcdOyaWf388wBhnyhjXWComo', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights'),
 Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_cfxmX91X6Cibza7GFed2l3aI', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_o6EBAFhc9xa3qvuMwSRe5s4c', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_wcdOyaWf388wBhnyhjXWComo'), ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_cfxmX91X6Cibza7GFed2l3aI'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_o6EBAFhc9xa3qvuMwSRe5s4c')], name=None)

You can define tools as pydantic models. To use pydantic model as tool, make sure the following points: 

- Write description of top level docstring field in the class. This is used for tool description.
- Define each argument fileds with description
- Implement run() method.

Tool definition using `ToolProperty`, `ToolParameter` and `ToolConfig` is almost equivalent to the following way. 

In [17]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field


class PowerDiscoBall(BaseModel):
    """Powers the spinning disco ball."""

    power: bool = Field(description="Boolean to spin disco ball.")

    def run(self) -> str:
        return f"Disco ball is {'spinning!' if self.power else 'stopped.'}"


class DimLights(BaseModel):
    """Dim the lights."""

    brightness: float = Field(description="The brightness of the lights, 0.0 is off, 1.0 is full.")

    def run(self) -> str:
        return f"Lights are now set to {self.brightness}"


class BPM(Enum):
    """The beats per minute of the music."""

    SLOW = "60"
    MEDIUM = "120"
    FAST = "180"


class StartMusic(BaseModel):
    """Play some music matching the specified parameters."""

    energetic: bool = Field(description="Whether the music is energetic or not.")
    loud: bool = Field(description="Whether the music is loud or not.")
    bpm: BPM = Field(description="The beats per minute of the music.")
    # bpm: Literal["60", "120", "180"] = Field(..., description="The beats per minute of the music.") # This also works

    def run(self) -> str:
        return f"Starting music! {self.energetic=} {self.loud=}, {self.bpm=}"

In [18]:
# Convert to ToolConfig

tools = [PowerDiscoBall, StartMusic, DimLights]
tool_configs_pydantic = [ToolConfig.from_pydantic(tool) for tool in tools]

In [19]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_configs=tool_configs_pydantic,
)

In [20]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively pace, and the lights are dimmed to create that cozy vibe. Enjoy the celebration! 🎉✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 150,
  'completion_tokens': 40},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Cro9uhjakFzVBdx93sqESdbd',
     'function': {'arguments': '{"power": true}', 'name': 'PowerDiscoBall'},
     'type': 'function'},
    {'id': 'call_sa9zI0GSBrlXlN3lbcwn7icK',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'StartMusic'},
     'type': 'function'},
    {'id': 'call_JL5SaO1pFq3rwnenG64tqGh7',
     'function': {'arguments': '{"brightne

Azure OpenAI Chat Completion 

In [21]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",  # your env variable name of OpenAI API key
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [22]:
response = chat_openai_azure.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The lights are dimmed, the disco ball is spinning, and the energetic music is pumping! The party mood is set, enjoy! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 135,
  'completion_tokens': 34},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_jwkBtd4OKFMFa3aEXAPdX6qX',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_CgpoUDKFN107vr9qNohbpxl5',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_HgSmALJzv5RDXm1bepMS7pjr',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'functi

In [23]:
response = await chat_openai_azure.arun(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Alright, the disco ball is spinning, the energetic music is blasting, and the lights are perfectly dimmed to set a comfortable party atmosphere. Time to hit the dance floor and enjoy the night!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 134,
  'completion_tokens': 39},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_0LhyPXAsrfTY2CjIav1Eq58w',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_vEi1OutCnbhRIZcOl8f6jXfG',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_vQODkzQc43Rk1Be8gncLgHCs',
     'function': {'argum

In [24]:
response = chat_openai_azure.run(prompt, tool_choice="start_music")

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've got the music going! Let's add some colorful lighting and some cozy seating areas to amp up the party mood. How about we set up a snack and drink station as well?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 64,
  'completion_tokens': 36},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_0LhyPXAsrfTY2CjIav1Eq58w',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_0LhyPXAsrfTY2CjIav1Eq58w',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=True, bpm='120'"}]}

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [25]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    # tools=[PowerDiscoBall, StartMusic, DimLights],
    # tool_configs=tool_configs_pydantic,
)

In [26]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, cranked up some energetic tunes, and dimmed the lights. Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 32},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
  

Asynchronous generation

In [27]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone, let's get this party started!  The disco ball is going, the music is pumping, and the lights are down low.  Time to have some fun! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 39},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value:

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [28]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  🎶 \n\nLet's get this party started! What else can I do to make it awesome? 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 68,
  'completion_tokens': 32},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! ener

### Gemini on VertexAI

In [29]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [30]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I've got the music going!  It's energetic but not too loud, and the lights are dimmed to create a nice party atmosphere.  The disco ball is spinning, throwing flashes of light all around.  \n\nWhat else can I do to make the mood even more comfortable?  Maybe you want some candles lit, or some snacks laid out?  Just tell me what you'd like, and I'll do my best to make this the perfect party spot! \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 73,
  'completion_tokens': 102},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fi

multiple generation

In [31]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Okay, I've set the mood!  \n\n* **Music:** Upbeat tunes with a good groove are playing (around 120 bpm). It's not too loud, perfect for easy conversation and dancing. \n* **Lights:** The lights are dimmed to a cozy level, creating a warm and inviting atmosphere. \n* **Disco Ball:** The disco ball is spinning, casting a playful sparkle around the room! \n\nLet the party begin! 🎉 \n"},
  {'text': "Okay, I've set the mood!  \n\n* **Music:** Upbeat tunes with a good groove are playing (around 120 bpm). It's not too loud, perfect for easy conversation and dancing. \n* **Lights:** The lights are dimmed to a cozy level, creating a warm and inviting atmosphere. \n* **Disco Ball:** The disco ball is spinning, casting a playful sparkle around the room! \n\nLet the party begin! 🎉 \n"}],
 'name': None}

### Anthropic Claude

tool_configs can be shared with Claude.

In [32]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    # tools=[PowerDiscoBall, StartMusic, DimLights],
    # tool_configs=tool_configs_pydantic,
)

In [33]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.4, which provides a cozy, intimate atmosphere without being too dark. This level of lighting is perfect for a comfortable party setting.\n\n2. Started the music: I've chosen energetic music to keep the party mood up, but set it to a lower volume (not loud) so people can still comfortably talk. The beats per minute (BPM) is set to 120, which is a good middle ground for various party music styles and allows for both dancing and casual listening.\n\n3. Activated the disco ball: The spinning disco ball will add a fun, party-like ambiance to the room without being overwhelming.\n\nThese settings should create a comfortable party mood that allows for both socializing and dancing. The combination of soft lighting, upbeat but not overpowering music, and the subtle disco ball effect should make for a welcoming

Asynchronous generation

In [34]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Powered on the disco ball: This will add a fun, party-like atmosphere with its spinning lights.\n\n2. Started some music: I've chosen energetic music to keep the party mood up, but not too loud so it's still comfortable for conversation. The tempo is set to 120 BPM, which is a good, moderate pace for a comfortable party vibe.\n\n3. Dimmed the lights: I've set the brightness to 0.6 (on a scale from 0 to 1). This creates a cozy atmosphere without making it too dark, allowing for a good balance between mood lighting and visibility.\n\nThese settings should create a comfortable party mood that's lively enough for fun but not overwhelming. The spinning disco ball adds a festive touch, the energetic music at a moderate volume keeps the energy up without drowning out conversation, and the dimmed lights create a relaxed ambiance.\n\nIs there anything else 

### Claude on Amazon Bedrock

In [35]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [36]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started some music with a moderate tempo (120 bpm). It's energetic to keep the party vibe going, but not too loud, so people can still comfortably chat.\n3. Turned on the disco ball to add a fun, festive touch to the space.\n\nThese settings should create a comfortable party mood that's perfect for socializing and having a good time. The combination of soft lighting, upbeat but not overwhelming music, and the subtle sparkle from the disco ball should set just the right tone.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable or to your liking?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 964,
  'completion_tokens': 194},
 'prompt': [{'role

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [37]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power']), type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm']), type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lights.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToo

Then these client tool config objects is transformed to client tools by format() method.

In [38]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}},
   'required': ['power']}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [39]:
from langrila.gemini.genai import GeminiToolConfig

In [40]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power'])),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm'])),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='brightness', type=

In [41]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For OpenAI Chat Completion

In [42]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [43]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The party mood is set! The disco ball is spinning, energetic music is playing at a loud volume with a lively beat of 120 BPM, and the lights are dimmed for a cozy atmosphere. Let's have a great time! 🎉🕺💃"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 134,
  'completion_tokens': 53},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Gl6lpeVbBVR9dB9FxDR6Vx5v',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2W7sF5UCEU94LC36LDX9LO0w',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_O5yhLKy6bPypTdxQPNj5Nrul',
     'fun

Next turn

In [44]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The music BPM has been cranked up to 180! Get ready to dance and enjoy the vibrant energy! 🎶💥'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 252,
  'completion_tokens': 26},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Gl6lpeVbBVR9dB9FxDR6Vx5v',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2W7sF5UCEU94LC36LDX9LO0w',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_O5yhLKy6bPypTdxQPNj5Nrul',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'}]},
  {'role': 'to

### For Gemini

In [45]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [46]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 30},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
      

In [47]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current music bpm is 120.  🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 185,
  'completion_tokens': 12},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }

In [48]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've bumped up the bpm to 180.  Let's get this party moving! 💃🕺 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 266,
  'completion_tokens': 27},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
          

## Multi-turn conversation using tools with multiple model

In [49]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [50]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at a lively tempo of 120 BPM, and the lights are dimmed to a cozy ambiance. Enjoy the celebration! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 134,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_hLmFv8iVKtn1oovTSWRX1qhE',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2xdCraXRkWXV20llSq9x2LYN',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_LXbEWUKtsUsPQWtOzqSLQbYR',
     'function': {'arguments': '

Claude can continue conversation without any additional process.

In [51]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The music has now been updated to a faster tempo of 180 BPM. This should create a more energetic and upbeat atmosphere for your party. The music is still energetic and loud, but now with a quicker beat that's perfect for dancing and keeping the energy high.\n\nIs there anything else you'd like to adjust to enhance the party mood further?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1122,
  'completion_tokens': 83},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_hLmFv8iVKtn1oovTSWRX1qhE',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_2xdCraXRkWXV20llSq9x2LYN',
     'type': 'tool_use',
     'input': {'energetic': True, 'loud': True, 'bpm': '120'},
     'name': 'start_music'},
    {'i

Go back to OpenAI Chat Completion

In [52]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music has been changed to a more relaxed vibe, playing at a calming 60 BPM. It's now softer and less energetic, perfect for creating a chill atmosphere. If you need any more changes or additions, just let me know!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 382,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_hLmFv8iVKtn1oovTSWRX1qhE',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2xdCraXRkWXV20llSq9x2LYN',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_LXbEWUKtsUsPQWtOzqSLQb

Gemini can follow as well

In [53]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Alright! The music is now back to a bpm of 120, keeping a relaxed vibe.  🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 572,
  'completion_tokens': 23},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {


In [54]:
prompt = "OK, then turn brightness to 0.7."

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The lights have been adjusted to a brightness level of 0.7. This should create a slightly brighter atmosphere than before, while still maintaining a comfortable ambiance for your relaxed party setting.\n\nTo summarize the current party setup:\n1. The disco ball is still spinning (from the initial setup).\n2. The music is playing at 120 BPM, with a relaxed (not energetic) and quieter volume.\n3. The lights are now at 70% brightness.\n\nIs there anything else you'd like to adjust to perfect your party atmosphere?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1663,
  'completion_tokens': 126},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_hLmFv8iVKtn1oovTSWRX1qhE',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'}

In [55]:
prompt = "Finally stop the disco ball."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The disco ball has been stopped. Your party atmosphere is now complete with the lights at 70% brightness and relaxing music at 120 BPM. If you need anything else or have any more adjustments in mind, just let me know! Enjoy the gathering! 🎉'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 739,
  'completion_tokens': 53},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_hLmFv8iVKtn1oovTSWRX1qhE',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2xdCraXRkWXV20llSq9x2LYN',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call

If you input non-relavant topic, any tool is not called.

In [56]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I'm unable to provide real-time weather updates. However, you can easily check the current weather by using a weather website, app, or asking a voice assistant with up-to-date information. If you have a specific location in mind, I can suggest how to look it up!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 810,
  'completion_tokens': 55},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_hLmFv8iVKtn1oovTSWRX1qhE',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_2xdCraXRkWXV20llSq9x2LYN',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'functi

Conversation is recorded with serialized universal message format.

In [57]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_hLmFv8iVKtn1oovTSWRX1qhE'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_2xdCraXRkWXV20llSq9x2LYN'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_LXbEWUKtsUsPQWtOzqSLQbYR'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_hLmFv8iVKtn1oovTSWRX1qhE',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_2xdCraXRkWXV20llSq9x2LYN',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_LXbEWUK

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [58]:
from langrila import InMemoryConversationMemory

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [59]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt, tool_choice="auto")

In [60]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [61]:
print(response.message.content[0].text)

The music BPM has been increased to **180**! Get ready for an even more energetic atmosphere! 🎶✨


In [62]:
response.usage

Usage(prompt_tokens=212, completion_tokens=23, total_tokens=235)

In [63]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_yxNy9lhcIqRD8dwTXcXeC0hy',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_puFIYV1GDn3y17P3cJ9s2syL',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'},
   {'id': 'call_nqgKm8xXDDgonQt9fYJrk19A',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_yxNy9lhcIqRD8dwTXcXeC0hy',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm='120'"},
 {'role': 'tool',
  'tool_call_id': 'call_puFIYV1GDn3y17P3cJ9s2syL',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 'tool',
  'tool_call_id': 'call_nqgKm8xXDDgonQt9fYJrk19A',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 

In [64]:
# whole history
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_yxNy9lhcIqRD8dwTXcXeC0hy'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_puFIYV1GDn3y17P3cJ9s2syL'},
   {'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_nqgKm8xXDDgonQt9fYJrk19A'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_yxNy9lhcIqRD8dwTXcXeC0hy',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_puFIYV1GDn3y17P3cJ9s2syL',
    'args': '{"brightness": 0.5}',
    'funcname': 'dim_lights'},
   {'output': 'Disco ball is spinning!',
    'call_id': 'call_nqgKm8xXD